## Parte 1

tenemos la posición y velocidad de cada grano de granizo expresada como:
    
x, y, z @ vx, vy, vz

las velocidades son constantes.

ignorando el eje z, cuantas trayectorias se cruzarán dentro de un área de prueba.

- test area de la prueba: 7, 27
- test area de la entrada 200000000000000, 400000000000000

In [25]:
def get_input(filename):
    granizo_map = []
    with open(filename) as file:
        for line in file:
            pos, vel = line.strip().split('@')
            granizo_map.append(tuple(int(p) for p in pos.split(',')) + tuple(int(v) for v in vel.split(',')))
    return granizo_map

In [27]:
def convert_to_explicit(line):
    """recibe una línea x, y, vx, vy y la convierte a y = mx + b"""
    x, y, _, vx, vy, _ = line
    m = vy / vx
    b = y - m * x
    
    return m, b

In [31]:
# test x = 3, y = 1, vx = 1, vy = -2
# y = -2x + 7
convert_to_explicit((3, 1, 0, 1, -2, 0))

(-2.0, 7.0)

In [43]:
def intersection(line1, line2):
    m1, b1 = line1
    m2, b2 = line2
    
    if m1 == m2:
        return (None, None)
    
    x = (b2 - b1) / (m1 - m2)
    y = m1 * x + b1
    
    return (x, y)

In [44]:
# test: y = 2x + 1; y = -3x + 6
# x = 1; y = 3
intersection((2, 1), (-3, 6))

(1.0, 3.0)

In [45]:
def get_tiempo(x0, x1, vx):
    return (x1 - x0) / vx

In [49]:
# test x0 = 3, vx = 1, x1 = 6
# t = 3
get_tiempo(3, 6, 1)

3.0

In [64]:
def contar_intersecciones(granizo_map, pmin, pmax):
    count = 0
    for i in range(len(granizo_map) - 1):
        mi, bi = convert_to_explicit(granizo_map[i])
        for j in range(i + 1, len(granizo_map)):            
            mj, bj = convert_to_explicit(granizo_map[j])
            
            x, y = intersection([mi, bi], [mj, bj])            
            if (x, y) == (None, None):
                continue
                
            xi_ini = granizo_map[i][0]
            vix = granizo_map[i][3]
            ti = get_tiempo(xi_ini, x, vix)
            
            xj_ini = granizo_map[j][0]
            vjx = granizo_map[j][3]
            tj = get_tiempo(xj_ini, x, vjx)                        
            
            if ti >= 0 and tj >= 0 and pmin <= x and x <= pmax and pmin <= y and y <= pmax:
                count += 1
    return count

In [65]:
granizo_map = get_input('test.txt')
intersecciones = contar_intersecciones(granizo_map, 7, 27)
intersecciones

2

In [66]:
granizo_map = get_input('input.txt')
intersecciones = contar_intersecciones(granizo_map, 200000000000000, 400000000000000)
intersecciones

27732

## Parte 2

encontrar la trayectoria para que tirando una piedra colisione con todos los granizos, la roca se tira en t = 0.

La roca no se relentiza ni cambia de trayectoria.

In [69]:
import sympy

In [115]:
def solve_symbolic(granizo_map):
    x, y, z, vx, vy, vz = sympy.symbols('x y z vx vy vz')
    ecuaciones = []
    tes = []
    
    #si hay choque y choca con 3 tiene que chocar con todas
    for i, g in enumerate(granizo_map[:3]):        
        x0, y0, z0, vx0, vy0, vz0 = g
        t = sympy.Symbol('t' + str(i))
        
        ecuacion_x = x + vx * t - x0 - vx0 * t
        ecuaciones.append(ecuacion_x)
        ecuacion_y = y + vy * t - y0 - vy0 * t
        ecuaciones.append(ecuacion_y)
        ecuacion_z = z + vz * t - z0 - vz0 * t
        ecuaciones.append(ecuacion_z)
        
        tes.append(t)
        
    resultado = sympy.solve_poly_system(ecuaciones, *([x, y, z, vx, vy, vz] + tes))
    return resultado[0][0] + resultado[0][1] + resultado[0][2]

In [117]:
granizo_map = get_input('test.txt')
resultado = solve_symbolic(granizo_map)
resultado

47

In [118]:
granizo_map = get_input('input.txt')
resultado = solve_symbolic(granizo_map)
resultado

641619849766168